# Markov Chain Monte Carlo (MCMC)

K. Leighly 2017

This lecture was drawn from the following sources:
 - Ivezic Chapter 5.8
 - Numerical Recipes Third Edition

## Motivation

There are several reasons why MCMC will be an attractive method to use for a variety of problems.

The first reason is motiviated by both the second homework problem in HW5, and Section 5.8 in Ivezic which is titled: "Numerical Methods for Complex Problems (MCMC)".  

Imagine that you are faced with an inference problem, but instead of having just a few parameters to estimate, you have many.  For example, some of the models I have been working with have 40 parameters, and I am certain that there will be cases when we are faced with 100 parameters.  Computing a grid of $\chi^2$, as you did in HW5, and searching for a minimum would be too computationally expensive, as the size of the grid increase dramatically.  

(This is called the "curse of dimensionality" and we will discuss it further when we talk about principal components analysis.)  

What you need to do is find a method that fairly samples parameter space, but doesn't need to compute it at every point.  Such a procedure is called **Monte Carlo**.  

Monte Carlo methods are good, but the problem with them is that parameteter space, as we note, is vast, and the Monte Carlo will generally spend a lot of computational energy in regions of parameter space where the $\chi^2$ is high.

In this case, you need a method that will _fairly_ sample parameter space, but will spend most of its time in regions where the $\chi^2$ is low (equivalently, the $\ln$ likelihood is high).  Such a process is called a **Markov Chain**.  

Together, these two methods form Markov Chain Monte Carlo (MCMC), and increasingly commonly-used method to sample the log likelihood.

## Example

Recall from the last lecture that we computed the likelihood surface as a function of parameters  that were constrained by the simulated data. A example is Ivezic Figure 5.10, which shows the log-likelihood contours for the Cauchy distribution for $\mu$ and $\gamma$, i.e., $L(\mu, \gamma)$, for $N=10$, where the sample was generated using $\mu=0$ and $\gamma=2$, reproduced below:

![Ivezic, Figure 5.10](http://www.astroml.org/_images/fig_likelihood_cauchy_1.png)

This figure was generated using the analytical form for the likelihood, given in Ivezic equation 5.75:

$$L_p = \log_e[p(\mu,\gamma |\{x_i\},I)]$$
$$ = \mbox{constant}+(N-1)\log_e \gamma -\sum_{i=1}^{N}
\log_e[\gamma^2 + (x_i-\mu)^2],$$

which itself is a function of $\mu$ and $\gamma$. So, given the simulated data $\{x_i\}$, at each choice of candidate $\gamma$ and $\mu$, the magnitude of the likelihood could be computed, and contours of given significance generated.

But what was required to produce this figure? _We needed to know the analytic form of the posterior distribution_. 

In addition, the fact that there are only two parameters to be determined makes a problem that can be illustrated as a two-dimensional contour and density plot.



What MCMC does for you is that it gives you a way to make this problem computationally tractable by sampling the multi-dimensional likelihood in an _ergodic_ way (so that it could, given enough samples, cover the whole parameter space), yet it builds up the most density in the regions of parameter space which are close to the maximum likelihood. Then, you can post-process the "chain" to infer the distribution and error regions. For example, Figure 5.22 below from Ivezic shows the same problem as  above, done with a Markov Chain Monte Carlo.

![Ivezic, Figure 5.22](http://www.astroml.org/_images/fig_cauchy_mcmc_1.png)

Here, the dashed contours show the results from the computation on a regular grid, as above, while the solid curves are the corresponding MCMC estimates using 10,000 sample points.  The left and bottom panels show the histograms of the  distributions for the two parameters.

## Markov Chain Monte Carlo Procedure Outline


- Starting at a random position, evaluate the likelihood.

- Choose a new position, according to some transition probabilities, and evaluate the likelihood there.

- Examine the odds ratio formed by the new-position likelihood and the old-position likelihood. If the odds ratio is greater than 1, move to the new position.  If it is less than one, keep it under the following conditions: draw a random number between zero and 1.   If the odds ratio is smaller than the random number, keep it. If not, reject the new position.
- Repeat 1-3 many times. After a period of time (the burn-in) the simulation should reach an equilibrium. Keep the results of the chain (after burn-in), and postprocess those results to infer the likelihood surface.


**Comment**: note that the goal when using MCMC is somewhat different from $\chi^2$ fitting. In the latter, our goal is to find the "best fitting value", and evaulate the uncertainty (at least sometimes) based on an expansion around that value. In MCMC, we are less concerned about the best fitting value (although we may still evaluate it), and more concerned about mapping as accurately as possible the potentially complicated likelihood space.



## Markov Chain Monte Carlo Basics

We will first discuss the basics of MCMC, including some motivation, and then delve into the different pieces in more or less detail. This discussion follows that in Numerical Recipes Third Edition.

First, let's discuss why MCMC is a Bayesian technique. The reason is that we are trying to constrain (sample) the posterior PDF of the data given the model. 

For example, the data might be a spectrum, and the model is a synthetic spectrum with lots of tunable parameters. The
posterior PDF will be the probability for particular values of the parameters. It will be multidimensional, but it can be projected upon any parameter by marginalizing over the other parameters. We don't know the posterior PDF, but we can obtain the likelihood (or $\chi^2$) of the model compared with the data for any sampled set of input parameters. The prior is likely to be a simple one, e.g., the constant over the range of the parameters of the model, where the range may be estimated based on physics.

As pointed out above, one could solve this problem with random sampling of the grid, but it can be computationally expensive to get the sampling density needed for the postprocessing. The advantage of the MCMC is that it will preferentially sample high probability regions of the grid.

The student of MCMC may be worried about this concept - if MCMC is going to preferentially sample high probability regions of the grid, how do we know that it is a fair sample? The reason that we are sure it is a fair sample comes about from the numerous conditions that are placed on the way the chain is produced. Some of these conditions include:

**The ergodic property**: The Markov Chain is ergodic, which means that it will eventually visit every point (say, $\mathbf{X}$, where the bold implies a vector of parameters) in parameter space with a probability proportional to the resulting PDF ($\pi(\mathbf{X}$)).  (Note: As noted by Gregory 2005, in the MCMC literature, it is common place to refer to our old friend $p(X|D,I)$ as $\pi(X)$. )



**The process is stationary and satisfies the detailed balance equation**: 

Several constraints are contained in this statement. 

First, there is the concept of a "Markov chain", which is simply a sequence of points with the property that each succeeding point depends _only_ on the previous point (i.e., $y_{i+1}$ depends only on $y_i$). Another way to say this is that it is _memoryless_. 

**Example of a process that is a Markov Chain** (from wikipedia):  

Suppose that you start with \$10, and you wager \$1 on an unending, fair, coin toss indefinitely, or until you lose all of your money. 

If $\mathbf{X}_n$ represents the number of dollars you have after $n$ tosses, with $\mathbf{X}_0=10$, then the sequence $\{\mathbf{X_n}\}$ is Markov process. For example, if I know that you have \$12 now, then it would be expected that with even odds, you will either have \$11 or \$13 after the next toss. _This guess is not improved by the added knowledge that you started with \$10, then went up to \$11, down to \$10, up to \$11, and then to \$12_.

Thus, what you have at any given point depends _only_ on the previous point.

**Example of a process that is not a Markov Chain** (also from wikipedia): 

Suppose that you have a coin purse containing five quarters (each worth 25c), five nickels (each worth 5c) and five dimes (each worth 10c), and one-by-one, you randomly draw coins from the purse and set them on a table. 

If $\mathbf{X}_n$ represents the total value of the coins set on the table after $n$ draws, with $\mathbf{X_0} = 0$, then the sequence $\{\mathbf{X_n}\}$ is not a Markov process (basically because you are drawing without replacement, so what you have at any draw depends on the history of _all_ previous draws, not just the one before.

_Stationary_ means that the process doesn't change in time, i.e., you get the same likelihood any time you vist the same point. 

The **detailed balance equation** is

$$\pi(\mathbf{x_1}) p(\mathbf{x_2} | \mathbf{x_1}) = \pi(\mathbf{x_2})
p(\mathbf{x_1} | \mathbf{x_2}),$$

where $\pi(\mathbf{x})=P(\mathbf{x} | \mathbf{D})$ is the posterior pdf, and $p$ is the transition probability function.  (Recall the detailed balance equation from stat mech - this equation does basically the same thing.)

This requirement is related to the idea that the process should be reversable, i.e., stationary, so that you might have started at $\mathbf{x_1}$ as well as starting at $\mathbf{x_2}$ and obtained the same answer. In addition, if those two points occur in proportion to the posterior probability, then the product of the PDF and the transition probability should be the same. A little thought will show that this condition and ergodicity are related, and are basically the same thing.

How will we ensure that this requirement is met? That will be the critical requirement of the method for choosing whether or not to keep the next point, or to discard it and choose again, i.e., the transition probability

We also need to be sure than the equilibrium distribution can be approached from any starting point, i.e., the resulting chain cannot depend on the initial value (although it will likely converge faster if you start near the final value).

Because of ergodicity, any point in the grid has a finite probability of being visited, even if that probability is very low. And recall that each position in the chain depends only on the previous position. So, if we start at a position $\mathbf{x_1},$ it is the same as starting at position $\mathbf{x_2}$, at some other point in the chain.

This can be understood in terms of a concept of "burn-in" - that if we start at an unlikely position in parameter space, the next point will also be unlikely, and the chain will take some time to settle down. So typically, the first $N$ points in the chain are thrown away, until the simulation settles down.

In practice, this may be somewhat difficult to pull off.  MCMC can get stuck in local minima.  So it is always best to start with reasonable guess for the starting point.

###  Sampling Algorithms

MCMC is the general concept; implementation can take many forms, in particular, in the choice of sampling algorithms.

A number of different sampling algorithms are available; we will talk about threeb:
 - Metropolis-Hastings (e.g., PyMC3)
 - Gibbs Sampler
 - affine-invariant ensemble sampler (e.g., emcee)
 
In class we will use the `emcee` method.  My collaborator likes the PyMC3 method.  The Gibbs method is supposedly good if you have a hierarchical bayesian problem.  Some methods can be combined, especially in the hierachical case.

### Metropolis-Hastings Algorithm

What transition probability function satisfies the detailed balance equation? One that does obey this requirement is the Metropolis-Hastings Algorithm. This is also the most common algorithm used in astronomical literature, although certainly `emcee` is quite commonly used too. 

The name comes from Metropolis, who realized the importance of the conditions above [Metropolis et al. 1953.](http://bayes.wustl.edu/Manual/EquationOfState.pdf), and Hastings, who came up with a transition probability function $p(\mathbf{x}_2 | \mathbf{x}_1)$ that satisfies the detailed balance equation.

Here is how the Metropolis-Hastings Algorithm works:

- Pick a proposal distribution $q(\mathbf{x_2|x_1})$. To quote numerical recipes "This can be pretty much anything you want, as long as a succession of steps generated by it can, in principle, reach everywhere in the region of interest."  For example $q(\mathbf{x_2|x_1})$ might be a multivariate normal distribution centered in $\mathbf{x_1}$.

     However, from my reading (e.g., there are some good illustrations in Gregory 2005), it seems that developing the   proposal distribution might be the challenging part of the exercise. If the proposal is too flat, the chain will take forever to converge, as there may be a lot of rejected points. If it is too peaked, it may accept lots of nearby points, but will if they are all of low probability, it may take a long time to burn-in.  Note that there are ways to modify this proposal distribution during the construction of the chain, e..g, by "tempering".

     Some discussion / illustration of this point can be found [here](http://twiecki.github.io/blog/2015/11/10/mcmc-sampling/).

- Generate a candidate point $\mathbf{x_{2c}}$ by drawing from the proposal distribution.

- Calculate an acceptance probability using
  $$\alpha(\mathbf{x_1},\mathbf{x_{2c}}) = min \left( 1,
\frac{\pi(\mathbf{x_{2c}})
q(\mathbf{x_1}|\mathbf{x_{2c}})}{\pi(\mathbf{x_1})
q(\mathbf{x_{2c}}|\mathbf{x_1})}\right).$$

  Remember that here, $\pi(\mathbf{x_1})$ is the posterior priority at $\mathbf{x_1}$.

- With probability $\alpha(\mathbf{x_1},\mathbf{x_{2c}})$, accept the candidate point and set $\mathbf{x_2} = \mathbf{x_{2c}}$, otherwise reject it and leave the point unchanged, i.e., $\mathbf{x_2}=\mathbf{x_1}$.

  The net result of this process is a transition probability:

  $$p(\mathbf{x_2}|\mathbf{x_1}) = q(\mathbf{x_2} | \mathbf{x_1})
\alpha(\mathbf{x_1},\mathbf{x_2}), \mbox{ when } \mathbf{x_2} \neq
\mathbf{x_1}.$$

It is shown in Numerical recipes that this procedure obeys detailed balance. It is also noted that this result can simplify, for example, if the proposal distribution $q$ is normal (apparently a common choice), then the ratio of $q(\mathbf{x_1} | \mathbf{x_{2c}})/ q(\mathbf{x_{2c}} | \mathbf{x_1})$ is just 1.  Then, the acceptance probability is related to  $[1, \pi(\mathbf{x_{2c}})/\pi(\mathbf{x_1})]$.

They note that there are other choices for $q$ that result in a similarly simple acceptance probability, such as a lognormal distribution.

In practice, the way this is done is that if the ratio of the likelihoods in step 3 is less than 1, then you draw a random number between zero and one. If the random number is less than the likelihood ratio, keep the new point, if it is greater than that, then set the new point to the old-point value.

_Why not throw away all new points that yield a likelihood ratio of less than 1?_ They seem to imply that you are going in the wrong direction. But remember, our goal is _not_ to get to the likelihood maximum, but map out the likelihood surface. So we need to keep some points that are less probable (but not all, because then we would not be choosing points at a rate proportional to the PDF).

### Gibbs Sampler

The Gibbs Sampler answers the question that the naive user might ask "Do I have to step all the parameters when I go from $\mathbf{x_1}$ to $\mathbf{x_{2c}}$? The answer, if you use the Gibb Sampler, is no, you can do them one at a time. That is, you choose one parameter, call it $x$, and vary it to the candidate position. Hold all the others, call them $\mathbf{x^-}$, fixed.

The acceptance probability becomes:

$$\alpha(x_1,x_{2c}|\mathbf{x^-}) =
min \left(1,
\frac{\pi(x_{2c}|\mathbf{x^-})q(x_1 |x_{2c},\mathbf{x^-})}
{\pi(x_1|\mathbf{x^-}) q(x_{2c}|x_1,\mathbf{x^-})}\right)$$

Choose a proposal distribution:

$$q(x_2|x_1,\mathbf{x^-}) = \pi(x_2|\mathbf{x^-}).$$

Substituting this in shows that the fraction in the equation above is equal to 1. That is, _the new point can always be accepted_.



The disadvantage is that the proposed transition probability must be properly normalized. 

Unlike the case above, where we assume the transition probability functional form is e.g., a multivariate Gaussian which can easily be normalized, this function isn't automatically normalized.

So the normalization factor $\int \pi(x|\mathbf{x^-})dx $ will have to be computed (probably numerically) at each step. 

Depending on the problem, this may be easy or hard. For example, it would probably be easy if the likelihoods along one direction are easily and quickly generated so that the numerical integral can be done.



### The Emcee Hammer

The Markov Chain Monte Carlo technique requires a sampler, i.e., a way to navigate parameter space. As we have already mentioned, the sampler must obey some mathematical properties, chiefly detailed balance, which ensures us that parameter space is fairly sampled. We have, so far, talked about the Metropolis Hastings sampler and the Gibbs sampler, but they are not the only ones known. There is a lot of buzz about the emcee Hammer, and we will discuss that here.

The webpage for emcee is [here](http://dan.iel.fm/emcee/current/), the paper describing the software is [here](http://adsabs.harvard.edu/abs/2013PASP..125..306F), and the paper describing the algorithm is [here](https://msp.org/camcos/2010/5-1/p04.xhtml).

(Note the number of citations on the software paper - today (September 16, 2017) there are 1205, while the last time this class was taught, there were, on September 26, 2015, 415.)




Some of the applications are quite interesting - for example, I was looking at the papers and noticed that a colleague of mine from way, way back, Chris Reynolds, referenced the software paper [here](http://adsabs.harvard.edu/abs/2015ApJ...808..154R). Could he be doing MCMC simulations?  It turns out no, or sort of.  [This package](https://github.com/jeremysanders/xspec_emcee) will allow the user to explore parameter space to produce covariance maps among parameters by running multiple threads of the commonly-used X-ray spectral fitting code XSPEC.   But it does turn out that emcee is included in the XSPEC software fitting package too, as discussed [here](http://xspector.blogspot.com/2013/02/xspec-v128-markov-chain-monte-carlo.html).

So [emcee is conquering the world!](http://www.gstatic.com/tv/thumb/movieposters/3274/p3274_p_v7_aa.jpg)


### Why emcee?

The reason that emcee is favored is that the Metropolis-Hastings algorthim (nice wikipedia article [here](https://en.wikipedia.org/wiki/Metropolis–Hastings_algorithm)) requires the choice of the proposal distribution or transfer function, in particular the width of the transfer function. How far away should the next step be? As we discussed previously, if the step is too small, it may take a long time to converge. If the step is too big, there may be many rejections. In either case, the simulation can become expensive.

The advantage of the `emcee` is that it employs a set of "walkers" that are something like individual chains in a Metropolis-Hastings simulation. The difference is the next step of a walker, instead of depending on its current position and the transition function, depends on the values of one or more of the other walkers, and all of the walkers are updated to the next point sequentially. (More details below). It seems that this enables the code to more quickly survey the parameter space. In this way, it may function better in situations where the posterior probability distribution is highly correlated.



[Here's](https://astrobites.org/2012/02/20/code-you-can-use-the-mcmc-hammer/) a brief description of the emcee Hammer.  And [here](https://vimeo.com/22616409) is an application to a difficult case (a parabolic function) using the Metropolis-Hastings algorithm. Notice how the sampler kind
of gets stuck in parts of the sampling.  ([Here](https://vimeo.com/22806729) is how it should look (using a different algorithm than emcee.) And [here](https://www.youtube.com/watch?v=yow7Ol88DRk) is an amusing take on the difference for a truly pathological posterior distribution.

One can see how the Metropolis-Hastings fails in these situations.  They ideally require a transition function that takes relatively long steps along the "ridge" but short steps perpendicular to it. But the M-H algorthim wants a fixed width of the transition function (although there are other implementations, like ones that use adaptive step size.)  [Here](https://healthyalgorithms.com/2011/01/28/mcmc-in-python-pymc-step-methods-and-their-pitfalls/) is further discussion of these difficulties.



## How does Emcee work?

As noted by Foreman-Mackey et al. 2013, the emcee is a "Python implementation of the affine-invariant ensemble sampler for Markov chain Monte Carlo (MCMC) proposed by Goodman & Weare (2010)."  To understand this algorithm, let's break down the statement above. Here, I am working from the Goodman & Weare 2010 paper.

**Affine-invariant**: For the Metropolis-Hastings algorithm, we drew the next step from a specified-width multi-dimensional Gaussian function. The `emcee` algorithm suggests that a better transition function is one that is affine, i.e., has the form $y=Ax+b$, i.e., a stretch and a shift.   See Figure 2 in Goodman & Weare (2010) for an illustration.  This has the property that:

$$\pi_{A,b}(y) = \pi_{A,b}(Ax+b) \propto \pi(x),$$

remembering that $\pi(x)$ is a short-cut form of $Prob(X|D,I)$.  That is, doing this transformation isn't going to change the posterior probability distribution.



As an example, consider the highly-covarient probability distribution function:

$$\pi(x) \propto
\exp \left(\frac{-(x_1-x_2)^2}{2\epsilon}+\frac{(x_1+x_2)^2}{2}\right).$$

See Figure 1 in Goodman & Weare 2010 for a plot of this PDF.

Metropolis or Gibbs would have to take steps of order $\sqrt{\epsilon}$ and would have very poor convergence. A better solution would have perturbations of order $\sqrt{\epsilon}$ in the $(1,-1)$ direction, and perturbations of order one in the $(1,1)$ direction.



Consider instead the affine transformation:

$$y_1=\frac{x_1-x_2}{\sqrt{\epsilon}} \mbox{, } y_2=x_1+x_2,$$

which turns the problem into:

$$\pi_A(y) \propto \exp(-(y_1^2+y_2^2)/2).$$

This is a simple two-dimensional normal distribution, and there would be no problem sampling it.



Consider a general MCMC sampler of the form $X(t+1)=R(X(t),\xi(t),\pi)$, where $X(t)$ is the sample after $t$ iterations, $\xi(t)$ is a uniformly distributed random variable between zero and 1 (typically), and $\pi$ is a probability density. 


An MCMC sampler will be affine invariant if, for any affine transformation $Ax+b$,

$$R(Ax+b,\xi(t),\pi_{A,b})= A R(x(t),\xi(t),\pi) +b.$$

More concretely, consider two Monte Carlo chains using the same random number generator and the same seed, so that the random numbers $\xi(t)$ will be the same for each one. 

Suppose that one of the runs uses probability density $\pi$ and starting point $X(0)$, and the other uses $\pi_{A,b}$ and initial point $Y(0)=AX(0)+b$.

If the algorithm is affine invariant, the sequencies will obey $Y(t)=AX(t)+b$, i.e., the one will be a stretch and shift of the other.  But they will sample the posterior PDF just as well.



#### Ensemble 

In the emcee algorithm, there is a family or ensemble of affine invariant walkers, with $x_k$ being walker $k$ in the ensemble of $L$ walkers.  The walkers are independent and drawn from $\pi$.  The probability density of $\mathbf{x}$, i.e., the ensemble, will then be:

$$\prod(\mathbf{x})=\prod(x_1,\ldots,x_L) = \pi(x_1) \pi(x_2)\ldots
\pi(x_L).$$

So the ensemble MCMC algorithm is a Markov chain "on the state space of ensembles". More concretely, initialized at $\mathbf{X(1)}$, it produces a sequence $\mathbf{X(t)}$.



### How it works

The brief version is that the new position of one walker is determined using a proposal generated with the help of the other walkers in the ensemble.  The bottom line seems to be that since they are all sampling the posterior PDF, it is ok to get information about the next step from other walkers.

Specifically, if we consider the walker $X_k$, its new position is generated based on the positions of the other walkers in the complementary ensemble (denoted by subscript $[k]$):

$$\mathbf{X_{[k]}(t)} = \{X_1(t+1),\ldots,X_{k-1}(t+1),X_{k+1}(t),\ldots,X_L(t)\}.$$

We denote then the transition kernal for moving $X_k$ as $\mu({dx_k},x_k|\mathbf{x_{[k]}})$. Some discussion is included in the paper about how this new position will preserve the posterior distribution (what we want) for the independent walkers.

In order to maintain detailed balance, the Metropolis Hastings rejection rule is used. The difference here is that using the information from other walkers leads the simulation to entertaining a bigger picture.



The simple stretch move is recommended, where the move looks like this:

$$X_k(t) \rightarrow Y = X_j + Z(X_k(t)-X_j),$$

where $X_j$ is a member of $\mathbf{X_{[k]}}$. This means that the new position is along the line connecting the old position and the randomly drawn $X_j$ position, but stretched by the factor $Z$. The scaling variable $Z$ itself is drawn from a distribution, and that distribution $g$ should satisfy

$$g\left(\frac{1}{z}\right) = z g(z).$$

They recommend

$$g(z) \propto \begin{cases} \frac{1}{\sqrt{z}} & \mbox{if }
\frac{1}{a} \leq z \leq a,\\ 0 & \mbox{otherwise},\end{cases}$$

where the parameter $a \ge 1$ can be tuned to improve performance. E.g., $a=2$ may work, so that the new position is between half as far and twice as far as the difference between the original and the randomly chosen $X_j$ from the original position.

(Note that $a$ is basically the only tunable parameter in `emcee`, with a default value of 2).

The acceptance probability turns out to be:

$$min \left\{1, Z^{n-1} \frac{\pi(Y)}{\pi(X_k(t))}\right\},$$

so that $X_k(t+1) = Y$ with this probability if this holds, or is set to $X_k(t+1)=X_k(t)$ otherwise.

Another option is for the new position to be based on a subset of $\mathbf{X_{[k]}}$, rather than a single randomly-drawn member of  $\mathbf{X_{[k]}}$.  A second option is the replacement move, where $X_k(t)$ is replaced by a sample from $\pi_S(x|S)$, where $X$ is the subset of $\mathbf{X_{[k]}}(t)$. See the Goodman & Weare 2010 paper for further discussion of these options.

## Example

All of that sounds pretty complicated; the implementation is much simpler.

As an example, we will simulate some data, and fit a straight line to it.  This example is taken from [here](http://dan.iel.fm/emcee/current/user/line/).  It is a straight line where the uncertainties have been underestimated.  

In [ ]:
%pylab inline

# Choose the "true" parameters.
m_true = -0.9594
b_true = 4.294
f_true = 0.534

# Generate some synthetic data from the model.
N = 50
x = np.sort(10*np.random.rand(N))
yerr = 0.1+0.5*np.random.rand(N)
y = m_true*x+b_true
y += np.abs(f_true*y) * np.random.randn(N)
y += yerr * np.random.randn(N)

In [ ]:
plt.xlabel('x')
plt.ylabel('y')

plt.errorbar(x,y,yerr=yerr,fmt='o')

We're going to fit this with a straight line, so we'll need some good starting points.  We can get those using just a polyfit, as previously done.  

But note that you don't need to use an analytic method, you can do it by eye by defining your model and varying parameters until it looks reasonable.

In [ ]:
result1=np.polyfit(x,y,1,w=1/yerr)

print 'The best fit coefficients ',result1

#poly1d will generate the model using the best fit coefficients.  
y1=np.poly1d(result1)
y1out=y1(x)
plt.errorbar(x,y, yerr=yerr, fmt='o')
plt.plot(x,y1out)


### Defining the model

The straight line has two parameters (note we are deviating from the webpage a bit here, to simplify things a little).  `emcee` requires the parameters to be defined in the model as a vector.  So we'll define param=[slope,intercept].

In [ ]:
def StraightLine(Param):
    OutSpectrum = Param[0]*x+Param[1]
    return OutSpectrum


### Likelihood


Remembering that MCMC is a Bayesian process, we need to define a likelihood (of the data given the model) and a prior.

For the likelihood, $\chi^2$ is a simple choice.  A more complicated likelihood is used on the example webpage.  


In [ ]:
def LnLike(Param,x,y,yerr):
     model=StraightLine(Param)
     inv_sigma2=1.0/(yerr**2)
     return -0.5*(np.sum(((y-model)**2)*inv_sigma2))


### Prior

For a problem like this, we hardly need a prior, as we don't have any prior information about the slope and the intercept.  But to see how it is done, we will define a prior.  Let's use a uniform prior with the slope and intercept constrained between likely values.

(I will show an example where prior is critical next Monday.)

In [ ]:
def LnPrior(Param):
    slopemax=0.0
    slopemin=-2.0
    interceptmax=8.0
    interceptmin=0.0
    if (slopemin <= Param[0] <= slopemax) and  (interceptmin <= Param[1] <= interceptmax):
        lnprobconst=0.0
    else:
        lnprobconst=-np.inf
    lnprior=lnprobconst
    return lnprior



### Posterior

Now we can write down the equation for the posterior probability distribution as:

$$p(M\,|\,D) = \frac{p(D\,|\,M)\,p(M)}{p(D)} \approx p(D\,|\,M)\,p(M)$$

Since we only care about the relative values of the posterior, we don't have to normalize it.

In [ ]:
def LnProb(Param,x,y,yerr):
    lp = LnPrior(Param)
    if not np.isfinite(lp):
        return -np.inf
    return lp + LnLike(Param,x,y,yerr)


Now we need to define the initial guess, and the initial positions of the walkers.  

We'll use the results from polyfit as the initial guess, and define the initial positions of the walkers in a small ball around that guess.  A small ball seems to work the best; they easily spread out within a few iterations.

We will use 100 walkers.  The more walkers the better, as long as your machine has adequate memory.

In [ ]:
slope=result1[0]
intercept=result1[1]
Param=np.zeros(2)
Param[0]=slope
Param[1]=intercept
print Param.shape

It is always a good idea to check your prior at the initial guess.  

In [ ]:

print LnPrior(Param)

Now set up your 100 walkers with suitable initial conditions.  As noted above, they should be in a small ball around the initial condition.

(Actually, because this is such a trivial implementation, I am specifying a relatively large ball; otherwise the simulation converges too rapidly.)

In [ ]:
nwalkers=100
ini_slope=slope+0.01*np.random.randn(nwalkers)
ini_intercept=intercept+0.1*np.random.randn(nwalkers)
print ini_slope.min(),ini_slope.max()
print ini_intercept.min(),ini_intercept.max()

In [ ]:
pos=[ini_slope]+[ini_intercept]
posNew=(np.array(pos)).T
print posNew[0]
print posNew.shape



It doesn't hurt to evaluate your initial positions with your prior. If the prior returns a -infty, then your simulation will not work.

In [ ]:
initial_lnprior=np.zeros(nwalkers)
for i in np.arange(100):
    paramtemp=posNew[i,:]
    initial_lnprior[i]=LnPrior(paramtemp)

print initial_lnprior.min()
print initial_lnprior.max()

### Import and run emcee

Now we have all the information we need to initialize and run emcee.

We will start by running 500 simulations


In [ ]:
ndim=Param.shape[0]
print ndim
numsims=500

import emcee
samplerNew = emcee.EnsembleSampler(nwalkers, ndim, LnProb, args=(x,y,yerr))
samplerNew.run_mcmc(posNew, numsims)


This illustrate the simplest implementation of the emcee.  Other useful parameters that can be specified are:
 - **threads**=some number, specified in emcee.EnsembleSampler.  This allows the computation to be parallelized, if you have a machine with multiple cores/threads.
 - **a**=some number greater than 1, specified in emcee.EnsemblerSampler.  $a$ is the scale factor mentioned above, and is the only tunable parameter in emcee.  The default value is 2, which should be find for most simple applications.
     - A small value of $a$ (e.g., 1.25) specifies a small stretch, which may cause the acceptance fraction to be high, but may take a long time to converge.
     - A large value of $a$ (e.g., 4) specifies a large stretch, which may be helpful in convergence, but may result in a low acceptance fraction.
 - **thin**=some integer, specified in .run_mcmc.  This saves a fraction of the steps to the chain, i.e., thin=10 saves every tenth value.  
     

samplerNew has lots of methods associated with it.  The details are [here](http://dan.iel.fm/emcee/current/api/).  Some of the most useful are:
 - the **chain** - a matrix with the dimenions (nwalkers,numsims,dim) which contains the results of the simulation.
 - **acceptance_fraction** - an array of length nwalkers that gives the acceptance fraction for each walker.  A value between 0.25 and 0.5 is desired.
 - **lnprobabilty** - the log probability evaluated at each point, so has the dimensions (nwalkers,nsims)
 
There are also methods that check the autocorrelation of the simulation.  We have not used that, but will be experimenting with it this semester.

Let's see how to use the chain and other properties.

In [ ]:
chain=samplerNew.chain
lnprob=samplerNew.lnprobability
acceptance=samplerNew.acceptance_fraction

print chain.shape
print lnprob.shape
print acceptance.shape

In [ ]:
plt.xlabel('Walker')
plt.ylabel('Acceptance Fraction')
plt.plot(acceptance)

The plot above shows that the acceptance fraction is very high for all walkers.  This probably speaks to the fact that this is a rather trivial application.

Next, we will look at the ln probability as a function of simulation number

In [ ]:
index=np.arange(numsims)

plt.axis([0,numsims,-470,-455])

plt.xlabel('Simulation Number')
plt.ylabel('Ln Prob')

for i in range(nwalkers):
    plt.plot(index,lnprob[i,0:numsims])


This plot shows that the simulation converges very rapidly.  The burnin is not very distinguishable.  Let's expand the x-axis to see it.

In [ ]:
plt.xlabel('Simulation Number')
plt.ylabel('Ln Prob')

plt.axis([0,50,-470,-455])
for i in range(nwalkers):
    plt.plot(index,lnprob[i,0:numsims])


### Burn-in

The burnin for this simulation, because it is so simple, is very rapid, only about ~30.   Usually I like to have enough simulations that I save only the last 1/2 of them.  So we'll set burnin=250, and save those, and also collapse the number-of-walkers vector.  We do this because all walkers are equivalent in mapping out the posterior probability distriubtion.  The result will be a matrix with size 250*nwalker, dim.



In [ ]:
burnin=250
ndim=chain.shape[2]

lnprob2=lnprob[:,burnin:].reshape((-1))
temp=chain[:,burnin:,0:ndim]
sample=temp.reshape(((numsims-burnin)*nwalkers,ndim))

print lnprob2.shape
print sample.shape

It can be instructive to plot a histogram of the lnprobability, especially if you want to compare models.

In [ ]:
numbins=100
lnprobmin=-470.0
lnprobmax=-455.0
lnprob_hist=np.array(np.histogram(lnprob2,bins=numbins,range=[lnprobmin,lnprobmax],density=1))
lnprob_hist_y=lnprob_hist[0]
temp=lnprob_hist[1]
binsize=temp[1]-temp[0]
lnprob_hist_x=lnprobmin+0.5*binsize+binsize*np.arange(numbins)
plt.xlabel('ln Prob')
plt.ylabel('Probability Density')
plt.plot(lnprob_hist_x,lnprob_hist_y,drawstyle='steps')
out=np.array([lnprob_hist_x,lnprob_hist_y])


Next, let's see what the individual parameters do as a function of simulation.  Note that one might want to wait until after this step to define the portion of the chain to be saved.

In [ ]:
pylab.rcParams['figure.figsize'] = (15, 6)
plt.xlabel('Simulation Number')
plt.ylabel('Slope')

for i in range(nwalkers):
    plt.plot(index,chain[i,0:numsims,0])


In [ ]:
plt.xlabel('Simulation Number')
plt.ylabel('Intercept')

for i in range(nwalkers):
    plt.plot(index,chain[i,0:numsims,1])


These show that convergence is very fast.  Nevertheless, we will set the burnin to 250.

We can make histograms of the vectors after removing the burnin.

In [ ]:
def MakeHistogram(samples, index, numbins):
    numout=len(samples)
    comp=samples[0:numout,index]
    comp_histogram=np.array(np.histogram(comp,bins=numbins,range=[comp.min(),comp.max()],density=1))
    comp_histogram_y=comp_histogram[0]
    temp=comp_histogram[1]
    binsize=temp[1]-temp[0]
    comp_histogram_x=comp.min()+0.5*binsize+binsize*np.arange(numbins)
    return comp_histogram_x,comp_histogram_y

print chain.shape
burnin=250
ndim=chain.shape[2]

lnprob2=lnprob[:,burnin:].reshape((-1))
temp=chain[:,burnin:,0:ndim]
sample=temp.reshape(((numsims-burnin)*nwalkers,ndim))

print lnprob2.shape
print sample.shape

In [ ]:
index=0
comp_hist_0_x,comp_hist_0_y=MakeHistogram(sample,index,numbins)
plt.xlabel('Slope')
plt.ylabel('Number')
plt.plot(comp_hist_0_x,comp_hist_0_y,drawstyle='steps')

In [ ]:
index=1
comp_hist_0_x,comp_hist_0_y=MakeHistogram(sample,index,numbins)
plt.xlabel('Intercept')
plt.ylabel('Number')
plt.plot(comp_hist_0_x,comp_hist_0_y,drawstyle='steps')

These are nice, well-behaved histograms.

From them we can obtain confidence intervals for the parameter values.

In [ ]:
percent_regions=[1.0-0.9973,1.0-0.9545,1.0-0.6827,0.5,0.6827,0.9545,0.9973]

confidence_regions_params=np.zeros([7,ndim])

numsamples=len(sample)

for i in np.arange(ndim):
    temp=sample[:,i]
    x2=np.sort(temp)
    f2=np.array(range(numsamples))/float(numsamples)
    confidence_regions_params[0:7,i]=np.interp(percent_regions,f2,x2)

print 'The median value of the slope is ',confidence_regions_params[4,0]
print 'The median value of the intercept is ',confidence_regions_params[4,1]

We can also get the density of the covariance of the two parameters and plot it.


In [ ]:
def PlotDensity(density, pos1out, pos2out):
    delta=density.max()/30.0
    levels=delta+delta*np.arange(30)
    plt.contourf(pos1out,pos2out,density,levels)

def MakeContour(samples, index1, index2, numbins):
     numout=len(samples)
     comp1=samples[0:numout,index1]
     comp2=samples[0:numout,index2]
     binsize1=(comp1.max()-comp1.min())/numbins
     binsize2=(comp2.max()-comp2.min())/numbins
     density=np.zeros([numbins,numbins])
     pos1=(comp1-comp1.min())/binsize1
     pos2=(comp2-comp2.min())/binsize2
     pos1out=comp1.min()+0.5*binsize1+binsize1*np.arange(numbins)
     pos2out=comp2.min()+0.5*binsize2+binsize2*np.arange(numbins)
     for i in range(numout):
        if (long(pos1[i])) < numbins and (long(pos2[i])) < numbins:
            density[long(pos1[i]),long(pos2[i])]=density[long(pos1[i]),long(pos2[i])]+1.0
     return density, pos1out, pos2out

index1=0
index2=1
numbins=100
density, pos1out, pos2out=MakeContour(sample,index1,index2,numbins)
PlotDensity(density,pos1out,pos2out)

Finally, we can get the median best fitting line and uncertainties on that by evaluating "y" at each "x" for all the simulations in "sample", then sorting and interpolating them.

In [ ]:
lenx=len(x)
numsamples=sample.shape[0]

line_sample=np.zeros([numsamples,lenx])

for i in range(numsamples):
    Param[0:ndim]=sample[i,0:ndim]
    OutSpec=StraightLine(Param)
    line_sample[i,0:lenx]=OutSpec[0:lenx]

confidence_regions=np.zeros([7,lenx])

for i in range(lenx):
    temp=line_sample[0:numsamples,i]
    x2=np.sort(temp)
    f2=np.array(range(numsamples))/float(numsamples)
    confidence_regions[0:7,i]=np.interp(percent_regions,f2,x2)




In [ ]:
pylab.rcParams['figure.figsize'] = (10, 10)
rc('axes',linewidth=2,labelweight='normal')
rc('xtick',labelsize='20')
rc('ytick',labelsize='20')
sp = 15
plt.xlabel('X',fontsize=24)
plt.ylabel('Y',fontsize=24)
#plt.axis([1080,1600,-0.10,1.20],fontsize=20,linewidth=4)
plt.errorbar(x,y,yerr=yerr,fmt='o')
plt.plot(x,confidence_regions[1,0:lenx],linewidth=2)
plt.plot(x,confidence_regions[5,0:lenx],linewidth=2)
plt.plot(x,confidence_regions[3,0:lenx],linewidth=2)


## Guidelines for implementation

Every problem is different, so it is likely difficult to generalize implementation. But in the reading, I found some general advice in Wall & Jenkins, and Gregory, and in other resources.



### Autocorrelation: 

Depending on the transition probability, there can be local correlations in the chain. That is, adjacent chain points can be quite correlated with one another, and therefore they are not necessarily ergodic and representative of the posterior probability distribution, although they aren't necessarily "bad", unless you are running short of memory.

How can you test for correlations? The correlation function is given by:

$$Corr(g,h) = \int_{-\infty}^{\infty} g(\tau+t)h(\tau)d\tau,$$
where this is written for the continuous case. For the discrete case that we are interested in here, the integral would be replaced by a summation. The autocorrelation is therefore
$$Corr(g,g) = \int_{-\infty}^{\infty} g(\tau+t) g(\tau) d\tau.$$

The idea is that you compute the autocorrelation function for some of your parameters, as a function of position in the chain. If there is low correlation, you will get a peak only when $t=\tau$, i.e., at a lag equal to zero. If there are correlations, then there will be significant autocorrelation for $t \neq \tau$. An example figure is in [this article](http://doingbayesiandataanalysis.blogspot.com/2011/11/thinning-to-reduce-autocorrelation.html).

Because of correlations in the chain (at least, each point is correlated with its neighbor), it is useful to thin out the chain by recording every $nth$ value, where $n$ can be, e.g., 10 or 100.
However, as pointed out in the article above, thinning might not be an efficient way to reduce autocorrelation, because you have to compute many more points in the chain in order to get a sufficently long thinned chain. (As noted above, if memory is the issue, thinning may still be useful).



### Acceptance Rates

Choosing the proposal well is key to efficient generation of the chain (this comment is for the traditional Metropolis-Hastings method). Acceptance rates of around 0.25-0.5 may give a good balance between correlation and excessive rejection. This means tuning the width of the proposal (if your proposal is a multivariate Gaussian).  Note that there are also adaptive methods that will vary the width of the proposal until burn-in has been achieved.

### Burn-in and Testing the Chain

How do you know how many points to exclude for burn-in? Likewise, could your solution get stuck in a local minimum? A way to address both these problems (to determine in general whether your chains are all right) is to analyze the chains. The idea is that ultimately, if the chain is truly mapping parameter space, it shouldn't matter where you start the chain; wherever you start, you should get the same result.

Some ways to systematically test your chains include splitting a chain in half (after excluding burn-in), and comparing, using e.g., standard deviation, the first half with the second half.  If the simulation has converged, the standard deviations should be the same.  At the same time, it is wise to compute multiple chains (also a good way to implement embarrassingly parrallel computing) and look at the standard deviation beween the different chains. In other words, look at (and analyze) your chain to see if you observe the behavior you expect, i.e., representative coverage of the posterior probability.

Apparently, burn-in can be slowed if there is correlation between variables. A change of variables (or changing to variables that are less correlated) may help.
